In [ ]:
def get_start_point(documents):
    s_point = 1
    for doc in documents:
        if len(re.findall(r'제 *1 *장', doc.text)) != 0 and (len(re.findall(r'목 *차', doc.text)) == 0 and len(re.findall(r'차 *례', doc.text)) == 0): 
            '''
            print(re.findall(r'제 *1 *장', doc.text))
            print(re.findall(r'목 *차', doc.text))
            print(re.findall(r'차 *례', doc.text))'''
            s_point = doc.metadata['page_label']
            break 
    return int(s_point) - 1

In [ ]:
def split_doc(idx, prev_spot, current_spot, document):
    file_name = document.metadata['file_name'].split('.')[0]
    splitted = document.text.split(current_spot)
    prev_doc = Document(text=splitted[0],
                       doc_id=f"{file_name}_doc_{idx}",
                       metadata={"spot": prev_spot, "file_name": document.metadata['file_name']},
                       excluded_llm_metadata_keys = ['spot', 'file_name']
                )
    current_doc = Document(text=splitted[1],
                       doc_id=f"{file_name}_doc_{idx + 1}",
                       metadata={"spot": current_spot, "file_name": document.metadata['file_name']},
                       excluded_llm_metadata_keys = ['spot', 'file_name']
                )
    return idx + 2, prev_doc, current_doc

In [ ]:
def get_doc_content(idx, document):  
    '''
    input: document (페이지 단위로 분할된 document object) 
    output: document_list (장 단위로 분할된 document objects) 
    '''
    doc_list = []; meta_info = dict();
    prev_spot = document.metadata['spot']
    split_spot = re.findall(r'제 *[0-9] *장', document.text)
    if len(split_spot) == 0:
        return document
        
    splitted_docs = [] 
    for spot in split_spot:
        new_idx, prev_doc, current_doc = split_doc(idx, prev_spot, spot, document)
        prev_spot = spot
        splitted_docs.append(prev_doc)
        document = current_doc
    return splitted_doc

In [ ]:
len(documents), s_point

In [ ]:
documents[s_point].metadata['spot'] = '제1장' 
for idx, doc in enumerate(documents):
    if idx >= s_point: 
        get_doc_content(idx, doc)

In [ ]:
documents[2].text.split('제2장')[1].split('제 3장')   #, documents[2].text.split('제2장')[1]